In [1]:
import pickle
import numpy as np, pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time, random
import os
from langdetect import detect

from PIL import Image

### Get all the urls for Must-read non-fiction:

In [ ]:
# # obtain url of books from each page of books listed
# urls_pages = []

# pages = list(range(1, 33))

# url_base = "https://www.goodreads.com/list/show/735.Must_Read_Non_Fiction?"

# for i in pages:
#     url_page = url_base + "page=" + str(i)
#     urls_pages.append(url_page)
# #urls_pages

In [ ]:
# # get all the urls ~3k must-read non-fiction
# urls_books = []
# gr_base = "https://www.goodreads.com"

# for i in urls_pages:
#     req = requests.get(i)
#     soup = BeautifulSoup(req.text, "lxml")
#     #maybe this is a good place to add a break
#     time.sleep(1+1.5*random.random())
    
#     for j in soup.find_all(class_="bookTitle"):
#         extension = j.get("href")
#         url_book = gr_base + extension
#         urls_books.append(url_book)

In [ ]:
# # save the urls
# with open('good_reads_url_list.pkl','wb') as pklfile:
#     pickle.dump(urls_books, pklfile)

### Define functions for scraping information for each book: 

In [ ]:
# function to clean up title info
def clean_text(title):
    text = re.sub("\n", "", title)
    text = text.strip()
    
    return text

In [ ]:
# create a function to extract the static data from the url:
def static_data(soup):
    #find the part of the html that contains data
    static = soup.find("div", class_="last col")
    
    #book title
    try:
        title = static.find(id="bookTitle").text
        title = clean_text(title)
    except:
        title = ""
    
    #book author(s)
    try:
        author = static.find("a", class_="authorName").text
    except:
        author = "" 
    
    #avg rating
    try:
        avg_rating = static.find("span", itemprop="ratingValue").text
        avg_rating = float(clean_text(avg_rating))
    except:
        avg_rating = ""
    
    #number of ratings
    try:
        num_rating = int(static.find_all("a", class_="gr-hyperlink")[0].find("meta").get("content"))
    except:
        num_rating = ""
    
    #number of reviews
    try:
        num_review = int(static.find_all("a", class_="gr-hyperlink")[1].find("meta").get("content"))
    except:
        num_review = ""
        
    #number of pages
    try:
        num_page = int(static.find("span", itemprop="numberOfPages").text.split()[0])
    except:
        num_page = ""
    
    #year published and publisher
    try:
        publish = static.find_all("div", class_="row")[1].text
        year = publish.split("\n")[2].strip()
        publisher = publish.split("\n")[3].strip()
    except:
        year = ""
        publisher = ""
        
    #putting all the information into a dictionary
    headers = ["title", "author", "rating", "num_rating", "review", "page", "year", "publisher"]
    book_data = dict(zip(headers, [title, author, avg_rating, num_rating, 
                                   num_review, num_page, year, publisher]))
    
    return book_data

In [ ]:
#create a function to extract the summary of the book
def book_summary(driver, soup, book_data):
    
    full_summary = None
    try:
        full_summary = driver.find_element_by_xpath("//div[contains(@id, 'descriptionContainer')]\
                                                     //a[contains(@onclick, 'swap')]")
        full_summary.click()
        time.sleep(3+2*random.random())
    except:
        pass

    if full_summary != None:
        html = driver.page_source
        soup = BeautifulSoup(html, "lxml") 
        summary = soup.find("div", id="descriptionContainer")
        summary = summary.find("span", style="").text
    else:
        try:
            summary = soup.find("div", id="descriptionContainer").find("span").text
        except:
            summary = ""
        
    book_data["summary"] = summary
    
    return book_data

In [ ]:
# create a function to save the book cover
def book_cover(driver, ind):
    try:
        enlarge_img = driver.find_element_by_xpath("//div[contains(@class, 'bookCoverContainer')]")
        enlarge_img.click()
        time.sleep(1+1.5*random.random())
    except:
        pass
    
    #get the url for the image
    img_html = driver.page_source
    img_soup = BeautifulSoup(img_html, "lxml")
    
    try:
        img_url = img_soup.find("div", id="box").find("img").get("src")
    except:
        img_url = ""
    
    if len(img_url) > 0:
        req = requests.get(img_url)
        file_ext = os.path.splitext(img_url)[-1]
    
        path = "/Users/sarazzzz/Desktop/Metis/CAMP/Metis_project5/book_images/book" + str(ind) + file_ext
    
        if req.status_code == 200:
            with open(path, 'wb') as f:
                for chunk in req:
                    f.write(chunk)
                
    #close the enlarge image window
    try:
        img_window = driver.find_element_by_xpath("//div[contains(@id, 'close')]")
        img_window.click()
    except:
        pass

In [ ]:
# read in the file that contains book urls 
# with open('good_reads_url_list.pkl','rb') as pklfile:
#     url_list = pickle.load(pklfile)
# with open('good_reads_biography_list.pkl','rb') as pklfile:
#     url_list2 = pickle.load(pklfile)

# for i in url_list[:10]:
#     print("BOOK URL:", i, "\n")
# len(url_list)

In [ ]:
additional = []
for i in url_list2:
    if i not in url_list:
        additional.append(i)
len(additional)    

In [ ]:
additional_df = pd.DataFrame(additional)
sum(additional_df.duplicated())

### Initialize chrome for scraping (in batch of 500 books):

In [ ]:
chrome_driver = "/Applications/chromedriver"
driver = webdriver.Chrome(chrome_driver)

In [ ]:
#########################################
# STOP!!! CHECK index before running code
#########################################
test = url_list[2500:]

master = []
 
start = time.time()
for ind, url in enumerate(test, 2501):
    #print(ind, url)
    driver.get(url)
    time.sleep(1+1.5*random.random())
    
    #extract static data
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    book_data = static_data(soup)

    #extract book summary
    book_data = book_summary(driver, soup, book_data)
    
    #aggregate data
    master.append(book_data)
    
    #extract book cover images
    book_cover(driver, ind)
    
print(time.time() - start)

In [ ]:
master_df = pd.DataFrame(master)
master_df.shape

In [ ]:
master_df.tail(20)

In [ ]:
master_df.to_pickle("data/sixth_692bks.pkl")

In [ ]:
df = pd.read_pickle("data/sixth_692bks.pkl")
df.tail()

In [ ]:
driver.close()

### Notes: Randomly chose 10 books from every 500 books to spot check summaries, looks good for most
### Will eliminate books missing summary

In [2]:
sixth = pd.read_pickle("data/sixth_692bks.pkl")
sixth.tail()

,title,author,rating,num_rating,review,page,year,publisher,summary
687,"True Story: Murder, Memoir, Mea Culpa",Michael Finkel,3.87,4652,515,,May 24th 2005,by HarperAudio,In the haunting tradition of Joe McGinniss's F...
688,The Spiral Staircase: My Climb Out of Darkness,Karen Armstrong,3.89,7824,647,306,February 22nd 2005,by Anchor,"In 1962, at age seventeen, Karen Armstrong ent..."
689,"Witness to the Revolution: Radicals, Resisters...",Clara Bingham,4.33,310,74,611,May 31st 2016,by Random House,"As the 1960s drew to a close, the United State..."
690,Postwar: A History of Europe Since 1945,Tony Judt,4.36,9984,763,933,September 5th 2006,by Penguin Books,"Almost a decade in the making, this much-antic..."
691,Mana Pounamu: New Zealand Jade,Russell J. Beck,3.33,3,0,184,November 28th 2002,by Reed Publishing (NZ) Ltd,Pounamu is the beautiful green stone that trad...


In [3]:
sample = np.random.choice(range(692), size = 10, replace=False)
print(sample)
sixth.iloc[sample]

[379 106 234 293 398 384 179 588  79 683]


,title,author,rating,num_rating,review,page,year,publisher,summary
379,On the Ragged Edge of Medicine: Doctoring Amon...,Patricia Kullberg,4.24,78,14,176,March 15th 2017,by Oregon State University Press,ON THE RAGGED EDGE OF MEDICINE offers a unique...
106,Miss Manners' Guide to Excruciatingly Correct ...,Judith Martin,4.28,1266,150,864,April 17th 2005,by W. W. Norton Company,Your neighbor denounces cellular telephones as...
234,How Many Lightbulbs Does It Take to Change a P...,Tony Juniper,3.67,24,2,256,April 5th 2007,by Quercus,Solution 12: Protest the most diverse and uniq...
293,"He's a Stud, She's a Slut, and 49 Other Double...",Jessica Valenti,3.89,3183,248,219,May 6th 2008,by Seal Press,Double standards are nothing new. Women deal w...
398,Shell Games: The Life and Times of Pearl McGil...,Jeffrey Copeland,4.20,10,1,354,February 18th 2012,by Paragon House,"Set against a backdrop of murder, intrigue, an..."
384,Upstairs at the White House: My Life with the ...,J.B. West,4.07,25028,2079,381,September 28th 1973,by Penguin Adult Hc/Tr,A fascinating behind-the-scenes look at life o...
179,The War Against Boys: How Misguided Feminism I...,Christina Hoff Sommers,3.92,1219,186,251,June 12th 2001,by Simon & Schuster,The author who aroused a storm of debate with ...
588,Running the Books: The Adventures of an Accide...,Avi Steinberg,3.51,4020,700,399,October 19th 2010,by Nan A. Talese,Avi Steinberg is stumped. After defecting from...
79,Neil Patrick Harris: Choose Your Own Autobiogr...,Neil Patrick Harris,3.82,33588,3711,291,October 14th 2014,by Crown Archetype,Tired of memoirs that only tell you what reall...
683,Accidental Saints: Finding God in All the Wron...,Nadia Bolz-Weber,4.15,10857,1255,,September 8th 2015,by Random House Audio,NEW YORK TIMES BESTSELLERWhat if that person y...


In [ ]:
for i in sixth.iloc[sample, -1]:
    print("BOOK SUMMARY:", i, "\n")

In [4]:
#load in previously scraped data and combine
first = pd.read_pickle("data/first_500bks.pkl")
second = pd.read_pickle("data/second_500bks.pkl")
third = pd.read_pickle("data/third_500bks.pkl")
fourth = pd.read_pickle("data/fourth_500bks.pkl")
fifth = pd.read_pickle("data/fifth_500bks.pkl")

In [5]:
all_bks = pd.concat([first, second, third, fourth, fifth, sixth]).reset_index(drop=True)

In [6]:
all_bks.tail()

,title,author,rating,num_rating,review,page,year,publisher,summary
3187,"True Story: Murder, Memoir, Mea Culpa",Michael Finkel,3.87,4652,515,,May 24th 2005,by HarperAudio,In the haunting tradition of Joe McGinniss's F...
3188,The Spiral Staircase: My Climb Out of Darkness,Karen Armstrong,3.89,7824,647,306,February 22nd 2005,by Anchor,"In 1962, at age seventeen, Karen Armstrong ent..."
3189,"Witness to the Revolution: Radicals, Resisters...",Clara Bingham,4.33,310,74,611,May 31st 2016,by Random House,"As the 1960s drew to a close, the United State..."
3190,Postwar: A History of Europe Since 1945,Tony Judt,4.36,9984,763,933,September 5th 2006,by Penguin Books,"Almost a decade in the making, this much-antic..."
3191,Mana Pounamu: New Zealand Jade,Russell J. Beck,3.33,3,0,184,November 28th 2002,by Reed Publishing (NZ) Ltd,Pounamu is the beautiful green stone that trad...


In [7]:
all_bks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3192 entries, 0 to 3191
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       3192 non-null   object 
 1   author      3192 non-null   object 
 2   rating      3192 non-null   float64
 3   num_rating  3192 non-null   int64  
 4   review      3192 non-null   int64  
 5   page        3192 non-null   object 
 6   year        3192 non-null   object 
 7   publisher   3192 non-null   object 
 8   summary     3192 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 224.6+ KB


In [8]:
# there are 106 books missing summary
mask = (all_bks["summary"] == "")
all_bks[mask].head()

,title,author,rating,num_rating,review,page,year,publisher,summary
88,"Father Forgive Us, for We Know Not What We Do",Helmut Glavar,4.41,145,8,,2020,,
134,Understanding China: Learning from China's Pas...,Stefan Piech,4.67,21,9,,September 19th 2020,,
144,Mary Me: One Woman’s Incredible Adventure with...,Elizabeth Bristol,4.37,297,40,346,June 16th 2020,,
298,Lift: Five Practices Great Managers Do Consist...,Raymond Wheeler,4.38,432,4,,June 20th 2020,,
351,A Script for Aspiring Women Leaders: 5 Keys to...,Mark Villareal,4.25,594,2,,September 23rd 2020,,


In [9]:
all_bks[mask].tail()

,title,author,rating,num_rating,review,page,year,publisher,summary
3023,An Essay Concerning Human Understanding,John Locke,3.85,14875,147,384,January 24th 2007,by Pomona Press,
3060,"Wilpon's Folly - The Story of a Man, His Fortu...",Howard Megdal,3.68,38,3,,,,
3098,De Geschiedenis van Rusland,Jaap ter Haar,3.21,14,1,235,1989,by Fibula,
3139,An Introduction to Aboriginal Societies,William H. Edwards,3.67,9,1,159,2004,by Socal Science Press,
3165,Een dorp in de polder. Graft in de zeventiende...,Arie Theodorus van Deursen,2.95,20,2,384,January 1995,by Bert Bakker,


In [10]:
# remove books without summary
df = all_bks.loc[all_bks["summary"] != ""]
df.shape

(3086, 9)

In [23]:
# some of the book summaries are not in English
lang = [detect(i) for i in df.loc[:, "summary"]]

# find out which books have non-English summary
non_eng = []
for ind, val in enumerate(lang):
    if val != "en":
        non_eng.append(ind)
print(non_eng)
print(len(non_eng))

['af', 'zh-cn', 'es', 'el', 'id', 'id', 'es', 'de', 'nl', 'nl', 'sl', 'it', 'es', 'sl', 'hr', 'de', 'ru', 'fr', 'tl', 'es', 'it', 'it', 'pt', 'it', 'pt', 'pt', 'pt', 'nl', 'nl', 'fr', 'nl', 'nl', 'nl', 'nl', 'nl', 'nl', 'nl', 'nl', 'nl', 'nl', 'nl', 'nl', 'nl']
43


In [24]:
# remove books with non-English summary from the data set
df.loc[:, "language"] = lang
mask = (df["language"] == "en")
df_eng = df[mask]
df_eng.shape

(3043, 10)

In [25]:
# save data
df_eng.to_pickle("data/bks_with_eng_summary.pkl")